In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [9]:
df = pd.read_csv('data/data_clean.csv', index_col=0, sep=',', header=0)

# cast timestamp to datetime format
df.timestamp = pd.to_datetime(df.timestamp)
df.head(2)

,timestamp,location,customer_id,date
0,2019-09-02 07:03:00,dairy,2019-09-02_1,2019-09-02
1,2019-09-02 07:03:00,dairy,2019-09-02_2,2019-09-02


### time each customer spent in the market:

In [4]:
minutes_spend_list = []

# loop through the customer id columnn:
for cust in df['customer_id'].unique():
    df_at = df[df['customer_id'] == cust] 
    
    # get first and last timestamp for current customer_id
    first_occ = df_at.timestamp[:1].values 
    last_occ = df_at.timestamp[-1:].values
    
    # calculate time diff
    timediff = (last_occ - first_occ)/60000000000 # nanoseconds to minutes
    
    minutes_spend_list.append(timediff.tolist()[0])


In [33]:
average = sum(minutes_spend_list) / len(minutes_spend_list)
max_time = max(minutes_spend_list)
min_time = min(minutes_spend_list)

def most_common(lst):
    return max(set(lst), key=lst.count)

most_common_time = most_common(minutes_spend_list)

print(
    'The average time customers spend in the supermarket is: \n', 
      round(average, 2), 'minutes \n', 
    'The maximum time customers spend in the supermarket is: \n',
      round(max_time, 2), 'minutes \n', 
    'The minimum time customers spend in the supermarket is: \n',
      round(min_time, 2), 'minute(s) \n', 
    'The most commom time customers spend in the supermarket is: \n',
      round(most_common_time, 2), 'minute(s)  \n', 
     )

The average time customers spend in the supermarket is: 
 6.26 minutes 
 The maximum time customers spend in the supermarket is: 
 52 minutes 
 The minimum time customers spend in the supermarket is: 
 1 minute(s) 
 The most commom time customers spend in the supermarket is: 
 1 minute(s)  



#### Pandas solution:

In [35]:
df_occurence = df.groupby(['customer_id', 'location']).agg({'timestamp': ['first', 'last']})
df_occurence.head()

timestamp                    
                                      first                last
customer_id    location                                        
2019-09-02_1   checkout 2019-09-02 07:05:00 2019-09-02 07:05:00
               dairy    2019-09-02 07:03:00 2019-09-02 07:03:00
2019-09-02_10  checkout 2019-09-02 07:08:00 2019-09-02 07:08:00
               fruit    2019-09-02 07:06:00 2019-09-02 07:06:00
2019-09-02_100 checkout 2019-09-02 08:03:00 2019-09-02 08:03:00

In [24]:
duration = df_occurence['timestamp','last'] - df_occurence['timestamp','first']

In [38]:
spent_minutes = duration.dt.seconds/60
spmi = spent_minutes.reset_index(name='time spent')
spmin = spmi.groupby('time spent').count()